In [582]:
import pandas as pd


%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [583]:
df = pd.read_csv("purchase_data.csv")

In [584]:
# player count

total_players = pd.DataFrame({df.SN.nunique()})
total_players.columns = ["Total Players"]
total_players

,Total Players
0,576


In [585]:
# purchasing Analysis (Total)

purchasing_analysis = pd.DataFrame(
    {
        "Number of Unique Items": [df["Item ID"].nunique()],
        "Average Price": "$" + format(df["Price"].mean(), ".2f"),
        "Number of Purchases": [df["SN"].count()],
        "Total Revenue": "$" + format(df["Price"].sum(), ".2f"),
    }
)

purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [586]:
# Gender Demographics

# Column index 0
g_count = df.groupby(df["Gender"])["SN"].nunique().sort_values(ascending=False)


# Column Index 1
total_count = df["SN"].nunique()


def gender_per(e):
    return e / total_count


percent_of_players = [gender_per(e) for e in g_count]


def per_stylized(e):
    return str(format(e * 100, ".2f")) + "%"


per_players = [per_stylized(e) for e in percent_of_players]


gender_dem = pd.DataFrame({"Total Count": g_count, "Percent of Players": per_players})
gender_dem

,Total Count,Percent of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [587]:
# Purchase Analysis (GENDER)

purchase_count = df.groupby(df["Gender"])["Purchase ID"].count()

g_avg_price = df.groupby(df["Gender"])["Price"].mean()

g_total_purchase = df.groupby(df["Gender"])["Price"].sum()

g_avg_total_purchase = (g_total_purchase / g_count).map("${:,.2f}".format)


gen_pur = pd.DataFrame(
    {
        "Purchase Count": purchase_count,
        "Average Purchase Price": g_avg_price,
        "Total Purchase Value": g_total_purchase,
        "Avg Total Purchase per Person": g_avg_total_purchase,
    }
)

dollar_format = "${:20,.2f}"

gen_pur.style.format(
    {"Average Purchase Price": dollar_format, "Total Purchase Value": dollar_format}
)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$ 3.20,$ 361.94,$4.47
Male,652,$ 3.02,"$ 1,967.64",$4.07
Other / Non-Disclosed,15,$ 3.35,$ 50.19,$4.56


In [588]:
# Age Demographics


df_unique = df[["Age", "SN"]].drop_duplicates()

bin_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

df_unique["age_counts"] = pd.cut(
    df["Age"],
    bins=[0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200],
    labels=bin_labels,
)


age_count = df_unique["age_counts"].value_counts()


def players(e):
    return e / total_count


per_players = [str(format(players(e) * 100, ".2f")) + "%" for e in age_count]


df_age = pd.DataFrame({"Total Count": age_count, "Percent of Players": per_players})

df_age.reindex(bin_labels)

,Total Count,Percent of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [589]:
# Purchasing Analysis (Age)


av_total = df.groupby(["SN"])["Price"].sum().to_frame()

new_purchase_df = pd.DataFrame.merge(df, av_total, on="SN")

age_counts = pd.cut(
    new_purchase_df["Age"],
    bins=[0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200],
    labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
)

age = pd.DataFrame({"age_ranges": age_counts})

purchase_analysis = pd.DataFrame.merge(
    new_purchase_df, age, left_index=True, right_index=True
)

purchase_analysis = purchase_analysis[["SN", "Price_x", "Price_y", "age_ranges"]]
purchase_analysis = purchase_analysis.groupby(["age_ranges"]).agg(
    {"Price_y": "mean", "Price_x": ["sum", "count", "mean"]}
)

purchase_analysis.columns = map("_".join, purchase_analysis)

purchase_analysis = purchase_analysis.rename(
    columns={
        "Price_y_mean": "Avg Total Purchase per Person",
        "Price_x_sum": "Total Purchase Value",
        "Price_x_count": "Purchase Count",
        "Price_x_mean": "Average Purchase Price",
    }
)

purchase_analysis.style.format(
    {
        "Avg Total Purchase per Person": dollar_format,
        "Total Purchase Value": dollar_format,
        "Average Purchase Price": dollar_format,
    }
)

,Avg Total Purchase per Person,Total Purchase Value,Purchase Count,Average Purchase Price
age_ranges,,,,
<10,$ 5.17,$ 77.13,23,$ 3.35
10-14,$ 4.30,$ 82.78,28,$ 2.96
15-19,$ 4.62,$ 412.89,136,$ 3.04
20-24,$ 5.32,"$ 1,114.06",365,$ 3.05
25-29,$ 4.94,$ 293.00,101,$ 2.90
30-34,$ 4.84,$ 214.00,73,$ 2.93
35-39,$ 5.59,$ 147.67,41,$ 3.60
40+,$ 3.34,$ 38.24,13,$ 2.94


In [590]:
# Top Spenders

purchase_count = df.groupby(["SN"]).agg({"Price": ["count", "mean", "sum"]})

purchase_count.columns = map("_".join, purchase_count)

purchase_count = purchase_count.rename(
    columns={
        "Price_count": "Purchase Count",
        "Price_mean": "Average Purchase Price",
        "Price_sum": "Total Purchase Value",
    }
).sort_values(by="Total Purchase Value", ascending=False)


purchase_count.head().style.format(
    {"Average Purchase Price": dollar_format, "Total Purchase Value": dollar_format}
)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$ 3.79,$ 18.96
Idastidru52,4,$ 3.86,$ 15.45
Chamjask73,3,$ 4.61,$ 13.83
Iral74,4,$ 3.40,$ 13.62
Iskadarya95,3,$ 4.37,$ 13.10


In [591]:
# Most Popular Items


most_pop = df.groupby(["Item ID", "Item Name", "Price"]).agg(
    {"Price": ["sum", "count"]}
)


most_pop.columns = map("_".join, most_pop)


most_pop = most_pop.rename(
    columns={"Price_count": "Purchase Count", "Price_sum": "Total Purchase Value"}
).sort_values(by="Purchase Count", ascending=False)


most_pop.reset_index().set_index(["Item ID", "Item Name"]).head().style.format(
    {"Price": dollar_format, "Total Purchase Value": dollar_format}
)

,,Price,Total Purchase Value,Purchase Count
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$ 4.23,$ 50.76,12
145,Fiery Glass Crusader,$ 4.58,$ 41.22,9
108,"Extraction, Quickblade Of Trembling Hands",$ 3.53,$ 31.77,9
82,Nirvana,$ 4.90,$ 44.10,9
19,"Pursuit, Cudgel of Necromancy",$ 1.02,$ 8.16,8


In [592]:
# Most Profitable Items


most_prof = df.groupby(["Item ID", "Item Name"]).agg({"Price": ["count", "sum"]})


most_prof.columns = map("_".join, most_pop)


most_prof = most_pop.rename(
    columns={"Price_count": "Purchase Count", "Price_sum": "Total Purchase Value"}
).sort_values(by="Total Purchase Value", ascending=False)

most_prof.reset_index().set_index(["Item ID", "Item Name"]).head().style.format(
    {"Price": dollar_format, "Total Purchase Value": dollar_format}
)

,,Price,Total Purchase Value,Purchase Count
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$ 4.23,$ 50.76,12
82,Nirvana,$ 4.90,$ 44.10,9
145,Fiery Glass Crusader,$ 4.58,$ 41.22,9
92,Final Critic,$ 4.88,$ 39.04,8
103,Singed Scalpel,$ 4.35,$ 34.80,8
